## 파이토치 기본

### 파이토치 공식 튜토리얼
- https://tutorials.pytorch.kr/

### 파이토치 패키지(API)
- `torch` : 메인 네임스페이스. 텐서(기본단위) 등 수학함수 포함. Numpy와 유사한 구조
- `torch.autograd` : 자동미분을 위한 함수들 포함. 컨텍스트 매니저, 기반클래스 포함
- `torch.nn` / `torch.nn.functional` : neuralnetwork. 신경망 구축 위한 데이터 구조, 레이어등이 정의되어 있는 모듈. ReLU 등도 포함
- `torch.optim` : SGD(확률적 경사 하강법) 중심 파라미터 최적화 알고리즘 포함
- `torch.utils` : SGD 반복 연산 시 배치용 유틸리티 함수 포함
- torch.multiprocessing : 파이토치용 병렬 프로세싱 환경을 안전하게 다루기 위한 모듈

### 텐서
- 파이토치 기본 단위
    - 1차원 배열 : 벡터
    - 2차원 배열 : 행렬(매트릭스)
    - 3차원 배열 : 텐서

- <img src="../image/ml0018.png" width="700">

- <img src="../image/ml0019.png" width="700">


In [878]:
import torch
import numpy as np

In [879]:
t1 = torch.tensor([1.0, 2.0, 3.0])
t1

tensor([1., 2., 3.])

In [880]:
# 텐서의 크기, 텐서 자료형, 자료형+전체 타입, 어느 디바이스에서 사용중 인지
print(t1.shape, t1.dtype, t1.type(), t1.device)

torch.Size([3]) torch.float32 torch.FloatTensor cpu


In [881]:
n1 = np.array([1.0, 2.0, 3.0])
n1

array([1., 2., 3.])

In [882]:
print(n1.shape, n1.dtype)

(3,) float64


In [883]:
## numpy에서 1, 0 으로 초기화되는 
n2 = np.ones([2,3])
n2

array([[1., 1., 1.],
       [1., 1., 1.]])

In [884]:
n3 = np.zeros([2,3])
n3

array([[0., 0., 0.],
       [0., 0., 0.]])

In [885]:
t2 = torch.ones([2,3])
t2

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [886]:
print(t2.shape, t2.dtype, t2.type(), t2.device)

torch.Size([2, 3]) torch.float32 torch.FloatTensor cpu


In [887]:
t3 = torch.zeros([2,3])
t3

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [888]:
# numpy로 만든 배열을 텐서로 변경
t4 = torch.from_numpy(n3)
t4

tensor([[0., 0., 0.],
        [0., 0., 0.]], dtype=torch.float64)

- 넘파이와 파이토치 텐서간의 형변환 아주 쉽다

In [889]:
t5 = torch.ones(3, 3, 3)
t5

tensor([[[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]]])

In [890]:
print(t5.shape, t5.dtype, t5.type(), t5.device)
print(t5.size())

torch.Size([3, 3, 3]) torch.float32 torch.FloatTensor cpu
torch.Size([3, 3, 3])


In [891]:
# 랜덤값, ???
t6 = torch.rand(2, 3)
t6

tensor([[0.1731, 0.8889, 0.7675],
        [0.6094, 0.9526, 0.5631]])

In [892]:
# cpu에 있는 텐서를 cuda로 이동
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [893]:
# t5(cpu)에 있는 텐서를 cuda로 변수명을 다르게 하면 복사, 변수명을 똑같이 하면 이동
t5 = t5.to(device=device)

In [894]:
print(t5.shape, t5.dtype, t5.type(), t5.device)

torch.Size([3, 3, 3]) torch.float32 torch.cuda.FloatTensor cuda:0


- cuda(GPU 사용) : 0(그래픽카드가 다수일 수 있음)
- 파이토치의 텐서 사용 == 넘파이 배열 사용

### GPU 사용법

In [895]:
# CPU사용
import torch.nn

# 신경망 모델 생성 : 3개의 입력을 넣어서 하나의 출력을 내는 신경망
model = torch.nn.Linear(3, 1)

sample_input = torch.tensor([[1.0, 2.0, 3.0]])
output = model(sample_input)
output

tensor([[1.8582]], grad_fn=<AddmmBackward0>)

In [896]:
# CPU사용
import torch.nn

# 신경망 모델 생성 : 3개의 입력을 넣어서 하나의 출력을 내는 신경망
model = torch.nn.Linear(3, 1)
model.to(device)

sample_input = torch.tensor([[1.0, 2.0, 3.0]]).to(device)
output = model(sample_input)
output

tensor([[-0.3972]], device='cuda:0', grad_fn=<AddmmBackward0>)

### PyTorch 모델 학습
- torch.nn.Module  로 신경망 모델 만들기 : 텐서플로우(함수)와 차이점 확인
- 손실 함수, 옵티마이저 설정학습
- 학습 루트 기본 구조 학습

#### 기본 구조 추가

In [897]:
# 파이토치 모듈 로드
import torch
import torch.nn as nn
import torch.nn.functional as F # 보통 F로 사용

# 모델 선언(클래스)
class SimpleNet(nn.Module): # nn.Module 클래스를 상속해서 클래스 생성
    def __init__(self):
        super(SimpleNet, self).__init__()
        # Linear : Dense Layer(케라스의 Dense()와 동일)
        self.fc1 = nn.Linear(4, 16) # input: 4차원, output: 16개 뉴런(차원)
        self.fc2 = nn.Linear(16, 3) # output 3개 차원 클래스(3가지 분류)

    def forward(self, x):   # 실제 연산이 수행되는 함수(자동 호출)
        x = F.relu(self.fc1(x)) # 첫번째 레이어 통과 : 활성화 함수(시그모이드, 소프트맥스, 렐루 모두 존재)
        x = self.fc2(x)        # 두번째 레이어 통화 : 출력층(필요 시 softmax 함수 사용)
        return x

- 텐서플로우(케라스)는 중간층에서 입력값을 설정하지 않아도 됨
- 파이토치는 입력값, 출력값을 모두 설정해야 함

#### 모델 생성

In [898]:
# 모델 확인
model = SimpleNet()
model

SimpleNet(
  (fc1): Linear(in_features=4, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=3, bias=True)
)

In [899]:
# 샘플 입력값
x = torch.rand(1, 4)
x

tensor([[0.3778, 0.3803, 0.3586, 0.7474]])

In [900]:
out = model(x)
out

tensor([[-0.1919, -0.1888,  0.3068]], grad_fn=<AddmmBackward0>)

#### 손실함수/옵티마이저 설정

In [901]:
# 분류(마지막 출력 3개) CrossEntropyLoss - 
criterion = nn.CrossEntropyLoss()

# 옵티마이저(Adam 권장)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01) # lr(learning rate)학습률 : 보통 0.01 ~ 0.0001 사이로 지정

#### 1회 Epoch 학습

In [902]:
y = torch.tensor([1])

# Forward
outputs = model(x)
loss = criterion(outputs, y)

# Backward(역전파) : 뉴런을 통과한 값이 다시 다음 훈련의 입력을 사용
# 기울기 초기화
optimizer.zero_grad() # 이전 단계 계산된 gradient(기울기/가중치) 초기화
# 역전파 계산
loss.backward() # 손실을 기준으로 각 파라미터에 대한 gradient 계산(오차 역전파)
# 파라미터 업데이트
optimizer.step() # 계산한 gradient를 이용, 파라미터 업데이터(학습훈련)

# 현재 손실값을 출력
loss.item()

1.2915515899658203

### 데이터셋, 데이터로더
- `Dataset`, `DataLoader` 학습
- 내장 데이터셋
- 커스텀 데이터셋 만들기
- 배치 학습 처리

#### 데이터셋, 데이터로더란?
- Dataset : 데이터를 불러오는 방법
- DataLoader : 데이터를 배치 단위로 나누기, 셔플, 병렬처리 방법

#### 추가
- 머신러닝, 딥러닝에 사용되는 데이터 샘플 종류 예제

In [903]:
from sklearn.datasets import load_breast_cancer, load_iris, load_wine # 11개 데이터셋
import seaborn as sns

datas = sns.load_dataset('titanic') # https://github.com/mwaskom/seaborn-data 확인

from torchvision.datasets import MNIST, FashionMNIST, CIFAR10 # 이미지 데이터셋
from keras.datasets import mnist, fashion_mnist, boston_housing, cifar10, cifar100, imdb # 이미지, 텍스트 데이터셋

In [904]:
datas

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [905]:
# 데이터셋 처리 모듈 로드
from sklearn.datasets import load_iris, load_wine
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader

In [906]:
# 데이터 로드
iris = load_iris()

## 입력(특성)데이터 X, 타겟 y == input, target
X = iris['data']
y = iris['target']

print(X.shape, y.shape) # numpy 데이터

(150, 4) (150,)


In [907]:
# sepal(꽃받침) length(cm), sepal width(cm), petal(꽃잎) length(cm), petal width(cm)
X[0, :]

array([5.1, 3.5, 1.4, 0.2])

In [908]:
# 0(setosa), 1(versicolor), 2(cirginica)
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [909]:
# 표준화(정규화)
scaler = StandardScaler() # 사이킷런의 표준정규화 클래스 사용
X = scaler.fit_transform(X)

In [910]:
X[0, :]

array([-0.90068117,  1.01900435, -1.34022653, -1.3154443 ])

In [911]:
# 훈련/테스트 세트 분리
train_scaled, test_scaled, train_target, test_target = train_test_split(X, y, test_size=0.2, random_state=42)

In [912]:
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)

In [913]:
print(train_scaled.shape, val_scaled.shape, test_scaled.shape)

(96, 4) (24, 4) (30, 4)


#### 커스텀 데이터셋 만들기

In [914]:
class IrisDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32) # 특성(입력)값은 실수
        self.y = torch.tensor(y, dtype=torch.long) # CrossEntropyLoss에서는 long타입 필요

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]

- 데이터셋으로 묶고, 데이터로더로 묶기

In [915]:
# 커스텀 데이터셋으로 생성 : 입력(특성)과 타겟을 하나로 묶음
train_dataset = IrisDataset(train_scaled, train_target)
val_dataset = IrisDataset(val_scaled, val_target)

In [916]:
# 데이터로더로 생성 : 배치학습, 셔플, 병렬 데이터 로딩
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)

- 실제 모델 훈련에서는 DataLoader를 사용

#### 배치 학습 반복

In [917]:
i = 1
for batch_input, batch_target in train_loader:
    outputs = model(batch_input)
    loss = criterion(outputs, batch_target)

    print(f'Vatch loss : {loss.item()} / {i}')
    i += 1

Vatch loss : 0.9662889242172241 / 1
Vatch loss : 1.1843750476837158 / 2
Vatch loss : 1.2985111474990845 / 3
Vatch loss : 1.2148231267929077 / 4
Vatch loss : 1.217618465423584 / 5
Vatch loss : 1.1988584995269775 / 6
Vatch loss : 1.0937002897262573 / 7
Vatch loss : 1.2697755098342896 / 8
Vatch loss : 1.3821921348571777 / 9
Vatch loss : 1.1375786066055298 / 10
Vatch loss : 1.0242289304733276 / 11
Vatch loss : 0.9757944345474243 / 12


- 96개 입력을 8으로 나눔 : 배치사이즈가 8이므로 12번 Epoch

### 학습 루프 작성
- 전체 학습 루프 구성
- Epoch, Batch 사용
- train(), evaluate() 확인

#### 신경망 클래스 정의

In [918]:
# 아이리스용 신경망
class IrisNet(nn.Module):
    def __init__(self):
        super(IrisNet, self).__init__()
        self.fc1 = nn.Linear(4, 16)         # fc1 : Full Connected Layer / tensorflow Dense()와 동일
        self.fc2 = nn.Linear(16, 3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

#### Iris 데이터 준비

In [919]:
# 커스텀 데이터셋 
class IrisDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=(torch.float32))
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]
    
iris = load_iris()
X = StandardScaler().fit_transform(iris.data)
y = iris.target

X_tarin, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Dataset, DataLoader 합쳐서 생성
train_loader = DataLoader(IrisDataset(X_tarin, y_train), batch_size=8, shuffle=True)
val_loader = DataLoader(IrisDataset(X_val, y_val), batch_size=8)


#### 모델 디바이스 전달

In [920]:
model = IrisNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [921]:
criterion

CrossEntropyLoss()

In [922]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.01
    maximize: False
    weight_decay: 0
)

#### 학습 루프 함수 / 평가 함수

In [924]:
# 학습 루프 함수
def train(model, dataloader, criterion, optimizer):
    model.train()
    total_loss = 0

    for X_batch, y_batch in dataloader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    return avg_loss

In [927]:
# 평가 함수
def evaluate(model, dataloader, criterion):
    model.eval() # 평가모드
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():   # 평가시에는 그레디언트 꺼줌
        for X_batch, y_batch in dataloader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            total_loss += loss.item()

            # 정확도
            preds = torch.argmax(outputs, dim=1)    # 제일 확률이 큰 값의 인덱스 (0, 1, 2)중 하나 값 선택
            correct += (preds == y_batch).sum().item()
            total += y_batch.size(0)

    avg_loss = total_loss / len(dataloader)
    accuracy = correct / total
    return avg_loss, accuracy

#### 훈련 실행

In [930]:
EPOCH = 20

for epoch in range(EPOCH):
    train_loss = train(model, train_loader, criterion, optimizer)
    val_loss, val_acc = evaluate(model, val_loader, criterion)

    print(f"[{epoch+1}/{EPOCH}] Train loss: {train_loss:.3f}, Val_loss: {val_loss:.3f}, Val_Accuracy: {val_acc:.2f}")

[1/20] Train loss: 0.374, Val_loss: 0.271, Val_Accuracy: 0.93
[2/20] Train loss: 0.337, Val_loss: 0.232, Val_Accuracy: 0.93
[3/20] Train loss: 0.291, Val_loss: 0.195, Val_Accuracy: 0.97
[4/20] Train loss: 0.248, Val_loss: 0.161, Val_Accuracy: 0.97
[5/20] Train loss: 0.202, Val_loss: 0.134, Val_Accuracy: 1.00
[6/20] Train loss: 0.169, Val_loss: 0.109, Val_Accuracy: 1.00
[7/20] Train loss: 0.140, Val_loss: 0.100, Val_Accuracy: 1.00
[8/20] Train loss: 0.123, Val_loss: 0.078, Val_Accuracy: 1.00
[9/20] Train loss: 0.106, Val_loss: 0.073, Val_Accuracy: 1.00
[10/20] Train loss: 0.102, Val_loss: 0.072, Val_Accuracy: 0.97
[11/20] Train loss: 0.096, Val_loss: 0.059, Val_Accuracy: 1.00
[12/20] Train loss: 0.086, Val_loss: 0.054, Val_Accuracy: 1.00
[13/20] Train loss: 0.083, Val_loss: 0.055, Val_Accuracy: 1.00
[14/20] Train loss: 0.083, Val_loss: 0.056, Val_Accuracy: 0.97
[15/20] Train loss: 0.073, Val_loss: 0.047, Val_Accuracy: 1.00
[16/20] Train loss: 0.073, Val_loss: 0.048, Val_Accuracy: 0.97
[

- CUDA로 0.7s